**What is a Decorator?**

A decorator is a function that takes another function as a parameter, adds extra behavior, and returns a new function - without modifying the original function directly.

They are commonly used for:
* Logging
* Authentication
* Measuring execution time
* Access control
* Caching

**Implementation**

In [2]:
def outer(func):
    def inner(*args, **kwargs):
        print("Decorator: start") 
        func(*args, **kwargs)
        print("Decorator: end")
    
    return inner


@outer
def sayHello(name):
    print(f"Hello {name}")

sayHello("Shamim")
# Expected output
# Decorator: start
# Hello Shamim
# Decorator: end

Decorator: start
Hello Shamim
Decorator: end


Write a add function then will always take two integer value and give sum of their absolute value

a =  2, b = 3,  a + b = 5 <br>
a = -2, b = 3,  a + b = 5 <br>
a = -2, b = -3, a + b = 5

In [8]:
def absolute(func):
    def inner(*args, **kwargs):
        new_args = [num if num > 0 else num * -1 for num in args]
        new_kwargs = {key: val if val > 0 else val * -1 for key, val in kwargs}
        result = func(*new_args, **new_kwargs)
        return result
    
    return inner


@absolute
def add(a: int, b: int) -> int:
    return a + b

inputs = [[2, 3], [-2, 3], [-2, -3]]

for a, b in inputs:
    print(add(a, b))

5
5
5


**Logging Decorator**

In [10]:
import datetime
import time


def custom_log(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        call_at = datetime.datetime.now() 
        print(f"{call_at} calling {func.__name__} with args: {args}, kwargs: {kwargs}") 
        result = func(*args, **kwargs)
        time.sleep(1)
        end = time.time()
        print(f"{func.__name__} executed within {end-start:.5f} seconds and returned {result}")
    return wrapper


@custom_log
def add(a, b):
    return a + b

add(5, 3)

2025-11-25 19:15:56.928804 calling add with args: (5, 3), kwargs: {}
add executed within 1.00118 seconds and returned 8


**Interview Question**

How would you write a decorator to check if a user is authenticated before executing an API handler?

1.Generic Python Version

In [11]:
def require_auth(func):
    def wrapper(user, *args, **kwargs):
        if not user.get("is_authenticated", False):
            raise PermissionError("User not authenticated")
        return func(user, *args, **kwargs)
    return wrapper

@require_auth
def get_profile(user):
    return f"Profile for {user['username']}"

In [13]:
user1 = {"username": "Shamim", "is_authenticated": True}
user2 = {"username": "Tamim", "is_authenticated": False}

In [16]:
print(get_profile(user1))

Profile for Shamim


In [15]:
print(get_profile(user2))

PermissionError: User not authenticated

2.FAST Api style

In [18]:
from fastapi import HTTPException, status

def require_auth(func):
    def wrapper(user, *args, **kwargs):
        if not user or not user.get("is_authenticated", False):
            raise HTTPException(status_code=status.HTTP_401_UNAUTHORIZED, detail="Not authenticated")
        return func(user, *args, **kwargs)
    return wrapper

@require_auth
def secure_endpoint(user):
    return {"message": f"Welcome {user['username']}"}

In [19]:
# Usage
print(secure_endpoint({"username": "shamim", "is_authenticated": True}))

{'message': 'Welcome shamim'}


In [20]:
# Usage
print(secure_endpoint({"username": "shamim", "is_authenticated": False}))

HTTPException: 401: Not authenticated